In [1]:
import pandas as pd
import requests
import datetime as dt
from typing import List
import matplotlib.pyplot as plt
import funcs.data_wrangling as dw

In [2]:
data = dw.ons_data(freq='h', ano_inicio=2000, ano_fim=2023, idreg="S")
df0, df = dw.pipeline(data)

c:\Users\user\Projetos\load_forecasting\lab\matheus\notebooks\funcs\data_wrangling.py:126: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  y.loc[:,"date"] = pd.to_datetime(y.loc[:,"date"])


In [4]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203026 entries, 0 to 203025
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        203026 non-null  datetime64[ns]
 1   id_reg      203026 non-null  object        
 2   desc_reg    203026 non-null  object        
 3   load_mwmed  202734 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 6.2+ MB


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 203040 entries, 2000-01-01 00:00:00 to 2023-02-28 23:00:00
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_reg      203040 non-null  object 
 1   desc_reg    203040 non-null  object 
 2   load_mwmed  203040 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.2+ MB


In [3]:
data = ons_data(freq='d', ano_inicio=2000, ano_fim=2023, idreg="S")
data.read()

In [4]:
data.check_date_column(printer=True)

Datas faltantes:
 []
Datas estranhas:
 [Timestamp('2013-02-02 00:00:01')]


In [13]:
data1 = dw.ons_data(freq='h', ano_inicio=2000, ano_fim=2023, idreg="S")
data1.read()
df1 = data1.data
data2 = dw.ons_data(freq='h', ano_inicio=2000, ano_fim=2023, idreg="S")
df2 = dw.pipeline(data2)

print('\n',df1.info())
print(df2.info())

Inserindo 14 datas.
Retirando 0 datas.
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 203026 entries, 2000-01-01 00:00:00 to 2023-02-28 23:00:00
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_reg      203026 non-null  object 
 1   desc_reg    203026 non-null  object 
 2   load_mwmed  202734 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.2+ MB

 None


c:\Users\user\Projetos\load_forecasting\lab\matheus\notebooks\funcs\data_wrangling.py:126: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  y.loc[:,"date"] = pd.to_datetime(y.loc[:,"date"])


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 203040 entries, 2000-01-01 00:00:00 to 2023-02-28 23:00:00
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_reg      203040 non-null  object 
 1   desc_reg    203040 non-null  object 
 2   load_mwmed  203040 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.2+ MB
None


In [21]:
df1.describe(include='all')

,id_reg,desc_reg,load_mwmed
count,203026,203026,202734.000000
unique,1,1,NaN
top,S,SUL,NaN
freq,203026,203026,NaN
mean,NaN,NaN,9296.426621
std,NaN,NaN,2560.562389
min,NaN,NaN,0.000000
25%,NaN,NaN,7408.465004
50%,NaN,NaN,9003.807495
75%,NaN,NaN,11154.261750
